In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage
from PIL import Image
from io import BytesIO
import requests
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')

In [2]:
model_client = OpenAIChatCompletionClient(model = "gemini-2.5-flash", api_key = api_key)

In [3]:
agent = AssistantAgent(
    name = "text_agent",
    model_client = model_client,
    system_message = 'You are a helpful assistant, answer the questions accurately'
)

In [5]:
async def test_text_messages():
    text_msg =TextMessage(content="Whay was Delhi choosen as the capital of India", source='user')
    result = await agent.run(task=text_msg)
    print(result.messages[-1].content)

await test_text_messages()

Delhi was chosen as the capital of British India in **1911**, replacing Calcutta (Kolkata). The decision was announced by King George V during the Delhi Durbar, based on the recommendation of Viceroy Lord Hardinge.

The reasons for this significant shift were multifaceted, combining strategic, historical, and administrative considerations:

1.  **Historical Significance and Symbolism:** Delhi has been the seat of power for numerous empires and kingdoms throughout Indian history, including the Delhi Sultanate and the Mughal Empire. It was laden with historical significance and strong symbolic resonance. Choosing Delhi connected the British Raj to a long line of powerful rulers, lending legitimacy and a sense of continuity to their rule in the eyes of the Indian populace.

2.  **Strategic Geographical Location:** Calcutta, while a major port and economic hub, was perceived as being on the periphery of the vast Indian subcontinent, located on the eastern coast. Delhi offered a more centra

### Multi-modal Messages

In [6]:
async def test_multi_modal():
    response = requests.get('https://picsum.photos/id/15/200/300')
    pil_image = Image.open(BytesIO(response.content))
    ag_image = AGImage(pil_image)

    multi_modal_msg = MultiModalMessage(
        content = ['What is in the image', ag_image],
        source='user'
    )

    result = await agent.run(task=multi_modal_msg)
    print(result.messages[-1].content)

await test_multi_modal()

The image shows a **tall, powerful waterfall** cascading down a steep, rocky gorge or cliff in a natural, lush environment.

Here's a breakdown of what's in the image:

*   **Waterfall:** The most prominent feature, a bright white stream of water, is falling vertically from a high point in the upper center of the image.
*   **Rocky Streambed/Foreground:** The lower two-thirds of the image are dominated by a streambed filled with numerous rocks, pebbles, and larger boulders. These rocks are of varying sizes and shades of grey, brown, and some appear wet, indicating the presence of water or recent flow. A shallow stream or pool of water can be seen winding through these rocks in the midground.
*   **Cliffs and Vegetation:** On either side of the waterfall, the cliffs are covered in dense, dark green foliage, including trees, mosses, and possibly ferns, suggesting a damp and verdant landscape. The rock faces themselves appear dark and rugged.

The overall impression is one of natural beau

### Unstructured Output

In [7]:
unstructured_model_client = OpenAIChatCompletionClient(
    model = 'gemini-2.5-flash',
    api_key = api_key
)

In [8]:
unstructured_agent = AssistantAgent(
    name = 'unstructured_planet_agent',
    model_client = unstructured_model_client,
    system_message = "You are a helpful assistant that provides information about planets in a structured json format"\
    "{name : str"\
    "age : int" \
    "}"
)

In [16]:
async def test_unstructured_output():
    task = TextMessage(content = "Please provide information about mars.", source='User')
    result = await unstructured_agent.run(task=task)
    unstructured_response = result.messages[-1].content
    return unstructured_response

unstructured_result = await test_unstructured_output()
print(unstructured_result)

```json
{
  "name": "Mars",
  "strage": 2
}
```


In [18]:
type(unstructured_result)

str

### Structured Output

In [19]:
from pydantic import BaseModel

class PlanetInfo(BaseModel):
    name: str
    color: str
    distance_miles: int

In [27]:
structured_model_client = OpenAIChatCompletionClient(
    model = 'gemini-2.5-flash',
    api_key = api_key,
    response_format = PlanetInfo,
)

In [28]:
structured_agent = AssistantAgent(
    name = "Structured_planet_agent",
    model_client = structured_model_client,
    system_message = "You are a helpful assistant that provides information about planets. in the JSON structure"
)

In [29]:
async def test_structured_output():
    task = TextMessage(content="Please provide information about mars.", source='User')
    result = await agent.run(task=task)
    structured_response = result.messages[-1].content
    return structured_response

structured_result = await test_structured_output()
print(structured_result)

Okay, let's dive into Mars, the captivating Red Planet!

Mars is the fourth planet from the Sun and one of the most studied objects in our solar system, largely because of its potential for past life and future human exploration.

---

### **Mars: The Red Planet in a Nutshell**

*   **Nickname:** "The Red Planet" due to iron oxide (rust) on its surface.
*   **Size:** About half the diameter of Earth, making it the second smallest planet.
*   **Atmosphere:** Very thin, mostly carbon dioxide.
*   **Temperature:** Extremely cold, averaging -63°C (-81°F).
*   **Water:** Abundant evidence of liquid water in its ancient past; mostly frozen water ice and subsurface ice today.
*   **Moons:** Two small, irregularly shaped moons, Phobos and Deimos.
*   **Key Features:** Polar ice caps, massive volcanoes (like Olympus Mons), vast canyons (like Valles Marineris), and extensive cratered terrain.
*   **Exploration:** The most explored planet besides Earth, with numerous orbiters, landers, and rovers